In [1]:
import sys
sys.path.append('../')

In [2]:
from tabnet.utils.logger import init_logger
from tabnet.estimator import TabNetRegressor
from sklearn.datasets import load_boston

In [3]:
logger_dir = 'logs'
logger_name = 'TestRegression'
level = 'INFO'

logger = init_logger(logger_dir=logger_dir, logger_name=logger_name, level=level)

In [4]:
X, y = load_boston(return_X_y=True)

print(X.shape)
print(y.shape)

(506, 13)
(506,)


In [5]:
tabnet = TabNetRegressor(
    input_dims=13, output_dims=[1], logger=None, is_cuda=False,
    reprs_dims=4, atten_dims=4, num_steps=3, num_indep=1, num_shared=1
)

In [6]:
tabnet.build(path=None)

TabNetRegressor(atten_dims=4, batch_size=1024, cate_dims=None,
                cate_embed_dims=1, cate_indices=None, criterions=['mse'],
                gamma=1.3, input_dims=13, is_cuda=False, is_shuffle=True,
                logger=None, mask_type='sparsemax', momentum=0.03, num_indep=1,
                num_shared=1, num_steps=3, num_workers=4, output_dims=[1],
                pin_memory=True, reprs_dims=4, task_weights=1,
                virtual_batch_size=128)

In [7]:
from torch.optim import Adam
from torch.optim import lr_scheduler


training_params = {
    'batch_size': 256,
    'max_epochs': 200,
    'metrics': ['mse'],
    'optimizer': Adam,
    'optimizer_params': {'lr': 0.1},
    'schedulers': [lr_scheduler.ExponentialLR],
    'scheduler_params': {'gamma': 0.99}
}


In [8]:
tabnet.fit(X, y.reshape(-1, 1), **training_params)

model :  <class 'tabnet.core._models.InferenceModel'>
data_loader :  <class 'torch.utils.data.dataloader.DataLoader'>
criterion :  <class 'tabnet.criterions._base._Criterion'>
optimizer :  <class 'torch.optim.adam.Adam'>
metrics :  <class 'list'>
logger :  <class 'NoneType'>
device :  <class 'str'>
is_train :  <class 'bool'>
model :  <class 'tabnet.core._models.InferenceModel'>
data_loader :  <class 'torch.utils.data.dataloader.DataLoader'>
criterion :  <class 'tabnet.criterions._base._Criterion'>
optimizer :  <class 'torch.optim.adam.Adam'>
metrics :  <class 'list'>
logger :  <class 'NoneType'>
device :  <class 'str'>
is_train :  <class 'bool'>
model :  <class 'tabnet.core._models.InferenceModel'>
data_loader :  <class 'torch.utils.data.dataloader.DataLoader'>
criterion :  <class 'tabnet.criterions._base._Criterion'>
optimizer :  <class 'torch.optim.adam.Adam'>
metrics :  <class 'list'>
logger :  <class 'NoneType'>
device :  <class 'str'>
is_train :  <class 'bool'>


KeyboardInterrupt: 

In [ ]:
importance, masks = tabnet.explain(X)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, axs = plt.subplots(4, 1, figsize=(5,15))

for i in range(3):
    axs[i].imshow(masks[i].cpu().numpy()[:20])
#     axs[i].set_xlabel('features')
    axs[i].set_ylabel('samples')
    axs[i].set_title(f"mask {i}")
    axs[i].set_yticks(range(20))
#     axs[i].set_xticks(range(30))
#     axs[i].set_xticklabels(feature_names, rotation=90)

axs[3].imshow(importance.cpu().numpy()[:20, :])
axs[3].set_xlabel('features')
axs[3].set_ylabel('samples')
axs[3].set_title('importance')
axs[3].set_yticks(range(20))
axs[3].set_xticks(range(13))
axs[3].set_xticklabels(feature_names, rotation=90)
plt.show()
axs[3].set_title('importance')